# Classifier

## Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm.notebook import tqdm, trange
from time import sleep
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from torchvision import transforms
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from collections import Counter
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import winsound


In [2]:
seed = 23
save_model = True
load_model = False

models_directory = "models"
metrics_directory = "metrics"
metrics_name = "metrics_filtered.csv"
dataset_name = "Fer2013_filtered_uniform_aug_big"
train_df_path = f"../datasets/{dataset_name}/train"
test_df_path = f"../datasets/{dataset_name}/test"

batch_size = 64

## Functions definition

In [3]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [4]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD, scheduler_func=None):
    history = []
    optimizer = opt_func(model.parameters(), lr, betas=(0.5,0.999))
    scheduler = scheduler_func(optimizer, mode='min', patience=3)
    for epoch in tqdm(range(epochs), desc = "Current Epoch"):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in tqdm(train_loader, desc = f"Epoch: {epoch}", leave= False):
            optimizer.zero_grad()
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()

        scheduler.step(result['val_loss'])
        result['lrate'] = get_lr(optimizer)

        model.epoch_end(epoch, result)
        history.append(result)
    return history

## Class definition

In [5]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [6]:
class Metric():
    def __init__(self, name):
        self.name = name
    
    def eval(self, outputs, labels):
        pass

In [7]:
class Accuracy(Metric):
    
    def eval(self, outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [8]:
class ImageClassificationBase(nn.Module):
    
    def __init__(self, loss_function, metrics):
        super().__init__()
        self.loss_function = loss_function
        self.metrics = metrics
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = self.loss_function(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = self.loss_function(out, labels)   # Calculate loss
        result = {'val_loss': loss.detach()}
        
        for m in self.metrics:
            result[m.name] = m.eval(out, labels)           # Calculate metrics
            
        return result
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        
        result = {'val_loss': epoch_loss.item()}
        
        for m in self.metrics:
            batch = [x[m.name] for x in outputs]
            epoch = torch.stack(batch).mean()      # Combine metrics
            result[m.name] = epoch.item()
            
        return result
    
    def epoch_end(self, epoch, result):
        out = f"Epoch [{epoch}]"
        vals = list(result.keys())
        for v in vals:
            out += f", {v}: {result[v]:.3e}"
        print(out)
        

In [9]:
class Net(ImageClassificationBase):
    
    def __init__(self, loss_function, metrics, out_size):
        super().__init__(loss_function, metrics)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2, padding=1)
        self.norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2, padding=1)
        self.norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.norm3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(in_features=128*5*5, out_features=256)
        #self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.fc = nn.Linear(256, out_size)

    def forward(self, input):
        output = self.norm1(self.pool1(F.relu(self.conv1(input))))
        output = self.norm2(self.pool2(F.relu(self.conv2((output)))))
        output = self.norm3(self.pool3(F.relu(self.conv3((output)))))
        output = output.view(-1, 128*5*5)
        output = F.relu(self.fc1(output))
        #output = F.relu(self.fc2(output))
        output = self.fc(output)
        output = F.softmax(input = output, dim=-1)
        return output

In [10]:
device = get_default_device()

## Dataset loading

In [11]:
df_train = ImageFolder(root=train_df_path, transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                     transforms.ToTensor()]))
df_test = ImageFolder(root=test_df_path, transform=transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                     transforms.ToTensor()]))

In [12]:
train_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size),device)
test_dl = DeviceDataLoader(DataLoader(df_test, batch_size=batch_size, shuffle=False),device)

In [13]:
classes = df_train.classes
classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [14]:
counts = dict(Counter(df_train.targets))
print(counts)
weights = np.array(list(counts.values()))
weights = torch.Tensor( min(weights)/weights)
print(weights)

{0: 3715, 1: 1421, 2: 3320, 3: 7867, 4: 4963, 5: 4679, 6: 3376}
tensor([0.3825, 1.0000, 0.4280, 0.1806, 0.2863, 0.3037, 0.4209])


## Model design and Training

In [15]:
loss_function = nn.CrossEntropyLoss(weight = weights)
metrics = []
optimizer = optim.Adam
scheduler = optim.lr_scheduler.ReduceLROnPlateau

lr = 0.0001
num_epochs = 20

In [16]:
splits = 4
skf = StratifiedKFold(n_splits=splits, random_state=seed, shuffle=True)
histories = []

for fold,(train_idx,val_idx) in enumerate(skf.split(df_train,train_dl.dl.dataset.targets)):
    print('------------fold no---------{}----------------------'.format(fold))
    
    net = Net(loss_function, metrics, len(classes))
    net.to(device)
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_fold_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size,  sampler=train_subsampler),device)
    val_fold_dl = DeviceDataLoader(DataLoader(df_train, batch_size=batch_size,  sampler=val_subsampler),device)
    #print(np.unique(np.array(train_fold_dl.dl.dataset.targets)[train_subsampler.indices],return_counts=True))
    #print(np.unique(np.array(train_fold_dl.dl.dataset.targets)[test_subsampler.indices],return_counts=True))
    #print(train_dl[train_subsampler.indices])
    history = dict()
    history["losses"] = fit(num_epochs, lr, net, train_fold_dl, val_fold_dl, optimizer, scheduler)
    history["model"] = net
    histories.append(history)

summary(net,(1,48,48))

------------fold no---------0----------------------


Current Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0:   0%|          | 0/344 [00:00<?, ?it/s]

Epoch [0], val_loss: 1.622e+00, train_loss: 1.718e+00, lrate: 1.000e-04


Epoch: 1:   0%|          | 0/344 [00:00<?, ?it/s]

...

Epoch [17], val_loss: 1.475e+00, train_loss: 1.244e+00, lrate: 1.000e-04


Epoch: 18:   0%|          | 0/344 [00:00<?, ?it/s]

Epoch [18], val_loss: 1.478e+00, train_loss: 1.241e+00, lrate: 1.000e-04


Epoch: 19:   0%|          | 0/344 [00:00<?, ?it/s]

Epoch [19], val_loss: 1.481e+00, train_loss: 1.238e+00, lrate: 1.000e-04
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 46, 46]             320
         MaxPool2d-2           [-1, 32, 23, 23]               0
       BatchNorm2d-3           [-1, 32, 23, 23]              64
            Conv2d-4           [-1, 64, 21, 21]          18,496
         MaxPool2d-5           [-1, 64, 11, 11]               0
       BatchNorm2d-6           [-1, 64, 11, 11]             128
            Conv2d-7            [-1, 128, 9, 9]          73,856
         MaxPool2d-8            [-1, 128, 5, 5]               0
       BatchNorm2d-9            [-1, 128, 5, 5]             256
           Linear-10                  [-1, 256]         819,456
           Linear-11                    [-1, 7]           1,799
Total params: 914,375
Trainable params: 914,375
Non-trainable params: 0
----------------------

## Save/Load model

In [17]:
if not os.path.isdir(models_directory):
    os.makedirs(name = models_directory)
models_directory_data = os.path.join(models_directory,dataset_name)
if not os.path.isdir(models_directory_data):
    os.makedirs(name = models_directory_data)

In [18]:
if save_model:
    for fold,h in enumerate(histories):
        model_path = os.path.join(models_directory_data,f"classifier_fold_{fold}.pt")
        torch.save(h["model"].state_dict(), model_path)

In [19]:
if load_model:
    histories = []

    for fold, model in enumerate(os.listdir(models_directory_data)):
        net = Net(loss_function, metrics, len(classes))
        net.load_state_dict(torch.load(os.path.join(models_directory_data,model)))
        net.eval()
        net.cuda()
        histories.append( {"model": net } )

## Model testing

In [20]:
def get_model_predicitons(model, dataset):
    y_test = []
    y_scores = []

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in tqdm(dataset):
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = model(images)

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)

            y_test.extend(labels.tolist())
            y_scores.extend(outputs.tolist())

    y_test = np.array(y_test)
    y_scores = np.array(y_scores)

    return y_test, y_scores


### Network Accuracy

In [21]:
def evaluate_accuracy(y_test, y_scores, classes, verbose=False):
    # Overall accuracy

    n_test = y_test.shape[0]
    predicted = np.argmax(y_scores,axis=1)
    mean_acc = np.sum(predicted == y_test) / n_test
    class_acc = []
    
    # Accuracy per class

    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # collect the correct predictions for each class
    for label, prediction in zip(y_test, predicted):
        if label == prediction:
            correct_pred[classes[label]] += 1
        total_pred[classes[label]] += 1

    for classname, correct_count in correct_pred.items():
        accuracy = float(correct_count) / total_pred[classname]
        class_acc.append(accuracy)

    if verbose:
        print(f'Overall accuracy: {100 * mean_acc } %')

        print("Accuracy per class:")
        for i,c in enumerate(classes):
            print(f'{c:9s} : {class_acc[i] * 100:.1f} %')
        
    return mean_acc, class_acc

# ROC/AUC

In [22]:

def evaluate_roc_auc(y_test, y_scores, classes, plot=False, verbose=False):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    ths = dict()
    roc_auc = dict()
    for i,_ in enumerate(classes):
        fpr[i], tpr[i], ths[i] = roc_curve(y_test == i, y_scores[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    if plot:
        # Plot of a ROC curve for a specific class
        for i,c in enumerate(classes):
            plt.figure()
            plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC {c}')
            plt.legend(loc="lower right")
            plt.show()

    mean_auc = np.mean(list(roc_auc.values()))

    if verbose:
        print(f"Mean AUC: { mean_auc * 100} %")

        print("Per classs AUC:")
        for i,c in enumerate(classes):
            print(f'{c:9s} : {roc_auc[i]*100:.2f} %')

    return mean_auc, roc_auc

In [23]:
y_test = dict()
y_scores = dict()

for fold,h in enumerate(histories):
    y_test[fold], y_scores[fold] = get_model_predicitons(h["model"],test_dl)

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [24]:
columns=["mean AUC", "angry AUC", "disgust AUC", "fear AUC", "happy AUC", "neautral AUC", "sad AUC", "surprise AUC", "mean ACC", "angry ACC", "disgust ACC", "fear ACC", "happy ACC", "neautral ACC", "sad ACC", "surprise ACC"]
df = pd.DataFrame(columns = columns)

for fold,_ in enumerate(histories):
    mean_acc,class_acc = evaluate_accuracy(y_test[fold], y_scores[fold], classes)
    mean_auc,roc_auc = evaluate_roc_auc(y_test[fold], y_scores[fold], classes)

    exp = [mean_auc]
    exp += list(roc_auc.values())
    exp += [mean_acc]
    exp += class_acc

    df_new = pd.DataFrame( np.array([exp]),[f"{dataset_name} fold {fold}"], columns=columns)
    df = pd.concat([df,df_new])
    df = df.astype('float')

mean_df = df.mean(0).to_frame().transpose().set_index(pd.Index([f"{dataset_name} mean"]))
df = pd.concat([df,mean_df])
df


,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_filtered_uniform_aug_big fold 0,0.909771,0.877283,0.919217,0.871600,0.959083,0.925900,0.840900,0.974417,0.634286,0.6400,0.5700,0.5000,0.7700,0.680,0.5100,0.770
Fer2013_filtered_uniform_aug_big fold 1,0.898987,0.857833,0.903583,0.843567,0.962025,0.912217,0.855467,0.958217,0.612857,0.5500,0.4200,0.5000,0.7600,0.670,0.6100,0.780
Fer2013_filtered_uniform_aug_big fold 2,0.902919,0.883217,0.936250,0.826383,0.954617,0.905567,0.842400,0.972000,0.594286,0.5300,0.4300,0.3900,0.8400,0.630,0.5700,0.770
Fer2013_filtered_uniform_aug_big fold 3,0.904810,0.870292,0.927233,0.831200,0.953833,0.918333,0.854867,0.977908,0.637143,0.5100,0.6100,0.4600,0.7800,0.680,0.6000,0.820
Fer2013_filtered_uniform_aug_big mean,0.904122,0.872156,0.921571,0.843187,0.957390,0.915504,0.848408,0.970635,0.619643,0.5575,0.5075,0.4625,0.7875,0.665,0.5725,0.785


## Save metrics

In [25]:
try:
    stored_df = pd.read_csv(os.path.join(metrics_directory, metrics_name),index_col=0)
except FileNotFoundError:
    stored_df = pd.DataFrame(columns = columns)
stored_df

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_filtered_uniform xval-4,0.904879,0.866508,0.940808,0.837265,0.960504,0.914504,0.840721,0.973844,0.614643,0.5175,0.4725,0.4700,0.8325,0.6375,0.5650,0.8075
Fer2013_filtered_uniform_aug xval-4,0.901435,0.873437,0.929525,0.833656,0.959258,0.902152,0.837410,0.974608,0.610714,0.5000,0.4900,0.4400,0.8200,0.5925,0.6525,0.7800
Fer2013_filtered_uniform_aug xval-4,0.908701,0.876058,0.931871,0.840271,0.965048,0.921031,0.855054,0.971575,0.626786,0.5750,0.5225,0.4475,0.8575,0.6050,0.5900,0.7900
Fer2013_filtered_uniform xval-4,0.900243,0.860883,0.936875,0.830688,0.957577,0.911533,0.833675,0.970469,0.607500,0.5550,0.4650,0.4550,0.8225,0.6000,0.5625,0.7925
Fer2013_filtered_uniform xval-4,0.902616,0.862852,0.947029,0.835208,0.953067,0.907823,0.838888,0.973448,0.603214,0.5050,0.4900,0.4300,0.8250,0.6200,0.5575,0.7950
Fer2013_filtered_uniform_aug xval-4,0.904633,0.874279,0.930833,0.837625,0.959083,0.923210,0.832917,0.974481,0.626429,0.5475,0.5325,0.4775,0.8275,0.6900,0.5250,0.7850
Fer2013_filtered_uniform_aug xval-4,0.906446,0.880850,0.935929,0.836500,0.958881,0.919258,0.840737,0.972962,0.625714,0.5500,0.5225,0.4450,0.8325,0.6800,0.5600,0.7900
Fer2013_filtered_uniform xval-4,0.908204,0.874727,0.949087,0.844775,0.954788,0.915271,0.846029,0.972754,0.624643,0.5400,0.4850,0.4925,0.8250,0.6675,0.5600,0.8025
Fer2013_filtered_uniform xval-4,0.904391,0.855921,0.945275,0.838765,0.953933,0.919619,0.848513,0.968712,0.615357,0.5300,0.5225,0.4725,0.8125,0.6675,0.5475,0.7550
Fer2013_filtered_uniform xval-4,0.905480,0.870017,0.940671,0.839137,0.955227,0.912417,0.851658,0.969231,0.619643,0.5375,0.4850,0.4900,0.8150,0.6375,0.5975,0.7750


In [26]:
stored_df = pd.concat([stored_df,mean_df.set_index(pd.Index([f"{dataset_name} xval-{splits}"]))])
stored_df

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Fer2013_filtered_uniform xval-4,0.904879,0.866508,0.940808,0.837265,0.960504,0.914504,0.840721,0.973844,0.614643,0.5175,0.4725,0.4700,0.8325,0.6375,0.5650,0.8075
Fer2013_filtered_uniform_aug xval-4,0.901435,0.873437,0.929525,0.833656,0.959258,0.902152,0.837410,0.974608,0.610714,0.5000,0.4900,0.4400,0.8200,0.5925,0.6525,0.7800
Fer2013_filtered_uniform_aug xval-4,0.908701,0.876058,0.931871,0.840271,0.965048,0.921031,0.855054,0.971575,0.626786,0.5750,0.5225,0.4475,0.8575,0.6050,0.5900,0.7900
Fer2013_filtered_uniform xval-4,0.900243,0.860883,0.936875,0.830688,0.957577,0.911533,0.833675,0.970469,0.607500,0.5550,0.4650,0.4550,0.8225,0.6000,0.5625,0.7925
Fer2013_filtered_uniform xval-4,0.902616,0.862852,0.947029,0.835208,0.953067,0.907823,0.838888,0.973448,0.603214,0.5050,0.4900,0.4300,0.8250,0.6200,0.5575,0.7950
Fer2013_filtered_uniform_aug xval-4,0.904633,0.874279,0.930833,0.837625,0.959083,0.923210,0.832917,0.974481,0.626429,0.5475,0.5325,0.4775,0.8275,0.6900,0.5250,0.7850
Fer2013_filtered_uniform_aug xval-4,0.906446,0.880850,0.935929,0.836500,0.958881,0.919258,0.840737,0.972962,0.625714,0.5500,0.5225,0.4450,0.8325,0.6800,0.5600,0.7900
Fer2013_filtered_uniform xval-4,0.908204,0.874727,0.949087,0.844775,0.954788,0.915271,0.846029,0.972754,0.624643,0.5400,0.4850,0.4925,0.8250,0.6675,0.5600,0.8025
Fer2013_filtered_uniform xval-4,0.904391,0.855921,0.945275,0.838765,0.953933,0.919619,0.848513,0.968712,0.615357,0.5300,0.5225,0.4725,0.8125,0.6675,0.5475,0.7550
Fer2013_filtered_uniform xval-4,0.905480,0.870017,0.940671,0.839137,0.955227,0.912417,0.851658,0.969231,0.619643,0.5375,0.4850,0.4900,0.8150,0.6375,0.5975,0.7750


In [27]:
if not os.path.isdir(metrics_directory):
    os.makedirs(name = metrics_directory)
stored_df.to_csv(os.path.join(metrics_directory, metrics_name))

In [28]:
stored_df.index.name = "Method"
stored_df.groupby("Method").mean()

,mean AUC,angry AUC,disgust AUC,fear AUC,happy AUC,neautral AUC,sad AUC,surprise AUC,mean ACC,angry ACC,disgust ACC,fear ACC,happy ACC,neautral ACC,sad ACC,surprise ACC
Method,,,,,,,,,,,,,,,,
Fer2013_filtered_uniform xval-4,0.904302,0.865151,0.943291,0.837640,0.955849,0.913528,0.843247,0.971410,0.614167,0.530833,0.486667,0.468333,0.822083,0.638333,0.56500,0.787917
Fer2013_filtered_uniform_aug xval-4,0.905754,0.874618,0.932903,0.837093,0.959521,0.917531,0.846415,0.972198,0.623929,0.543750,0.516250,0.460312,0.821250,0.655625,0.57625,0.794063
Fer2013_filtered_uniform_aug_big xval-4,0.904828,0.871261,0.919433,0.842030,0.960259,0.918906,0.850027,0.971876,0.627500,0.528750,0.521250,0.476250,0.806250,0.675000,0.59750,0.787500


In [29]:
winsound.Beep(400, 2000)

In [30]:
winsound.Beep(400, 2000)